In [2]:
import pandas as pd  
import nltk
import gensim
from time import time

In [3]:
train = pd.read_csv('train.csv', index_col='id')
train.head(3)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
id,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated


#### Data Pre-Processing

In [0]:
# removing everything except alphabets`
train['title1_en_cleaned'] = train['title1_en'].str.replace("[^a-zA-Z#]", " ")
train['title2_en_cleaned'] = train['title2_en'].str.replace("[^a-zA-Z#]", " ")

# removing short words
train['title1_en_cleaned'] = train['title1_en_cleaned'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train['title2_en_cleaned'] = train['title2_en_cleaned'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
train['title1_en_cleaned'] = train['title1_en_cleaned'].apply(lambda x: x.lower())
train['title2_en_cleaned'] = train['title2_en_cleaned'].apply(lambda x: x.lower())

In [0]:
train.head()

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label,title1_en_cleaned,title2_en_cleaned
id,,,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated,there insurance benefits people rural areas ha...,police disprove bird nest congress each person...
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated,come shenzhen sooner later your will also come...,shenzhen outstrips hong kong shenzhen statisti...
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated,come shenzhen sooner later your will also come...,overtopped hong kong shenzhen clarified little...
2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated,come shenzhen sooner later your will also come...,shenzhen topped hong kong last year shenzhen b...
9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed,discriminate from gutter means garlic,took years cooking know that piece garlic easy...


In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# tokenization
train['title1_en_cleaned'] = train['title1_en_cleaned'].apply(lambda x: x.split())
train['title2_en_cleaned'] = train['title2_en_cleaned'].apply(lambda x: x.split())

# remove stop-words
train['title1_en_cleaned'] = train['title1_en_cleaned'].apply(lambda x: [item for item in x if item not in stop_words])
train['title2_en_cleaned'] = train['title2_en_cleaned'].apply(lambda x: [item for item in x if item not in stop_words])



In [0]:
train.head()

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label,title1_en_cleaned,title2_en_cleaned
id,,,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated,"[insurance, benefits, people, rural, areas]","[police, disprove, bird, nest, congress, perso..."
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated,"[come, shenzhen, sooner, later, also, come, le...","[shenzhen, outstrips, hong, kong, shenzhen, st..."
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated,"[come, shenzhen, sooner, later, also, come, le...","[overtopped, hong, kong, shenzhen, clarified, ..."
2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated,"[come, shenzhen, sooner, later, also, come, le...","[shenzhen, topped, hong, kong, last, year, she..."
9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed,"[discriminate, gutter, means, garlic]","[took, years, cooking, know, piece, garlic, ea..."


In [0]:
def dekotenize(text):
    import re
    from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
    detokenizer = Detok()
    text = detokenizer.detokenize(text)
    text = re.sub('\s*,\s*', ', ', text)
    text = re.sub('\s*\.\s*', '. ', text)
    text = re.sub('\s*\?\s*', '? ', text)
    return text

In [0]:
# Standard modules import
import sys
import re

In [0]:
train['title2_en_cleaned'] = train['title2_en_cleaned'].apply(dekotenize)
train['title1_en_cleaned'] = train['title1_en_cleaned'].apply(dekotenize)

In [0]:
train.head(1)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label,title1_en_cleaned,title2_en_cleaned
id,,,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated,insurance benefits people rural areas,police disprove bird nest congress person gets...


In [4]:
import multiprocessing
from time import time
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=0,
                     workers=cores-1)

In [0]:
text = train['title1_en_cleaned']
text_corpus = text.str.cat(sep=' ')

In [6]:
text1 = train['title1_en']
text2 = train['title1_en']
text_corpus = text1.str.cat(sep=' ')+text2.str.cat(sep=' ')

In [7]:
t = time()

w2v_model.build_vocab(text_corpus, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.05 mins


In [ ]:
t = time()

w2v_model.train(text_corpus, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [16]:
# similarity between two different titles
w2v_model.wv.similarity('insurance', 'sooner')

KeyError: "word 'insurance' not in vocabulary"

In [12]:
vector = w2v_model.wv['insurance'] 

KeyError: "word 'insurance' not in vocabulary"